In [1]:
import os
import pandas as pd
import torch
import numpy as np

from torch.utils.data import Dataset
from skimage import io

In [2]:
def from_string_to_label(object_name:str):
    label_map = {
        'big_drone': 0,
        'bird': 1,
        'free_space': 2,
        'human': 3,
        'small_copter': 4
    }
    
    return label_map.get(object_name, -1)

In [3]:
cur_folder_path = os.getcwd()

os.listdir(cur_folder_path)

['dataset.json',
 'resnet34_model_4.pth',
 'resnet34_model_5_87.pth',
 'resnet34_model_2.pth',
 '.ipynb_checkpoints',
 'google_model_1.pth',
 'dataset_shan.json',
 'download_dataset.ipynb',
 'green.png',
 'resnet34.pth',
 'net.ipynb',
 'resnet34_model_3.pth',
 'resnet34_model_1.pth',
 'resnet34_model.pth']

In [4]:
import json
from PIL import Image

def load_json(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

In [5]:
class DroneDataset(Dataset):
    def __init__(self, json_file, root_dir, transform=None):
        json_path = os.path.join(root_dir, json_file)
        self.data = load_json(json_path)
        self.transform = transform
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        
        image_arr = np.array(self.data[index].get('coefs'))
        image = Image.fromarray(image_arr, mode='L').convert('RGB')
        
        object_label = from_string_to_label(self.data[index].get('object'))
        y_label = torch.tensor(int(object_label))
        
        if self.transform:
            image = self.transform(image)
            
        return (image, y_label)

In [6]:
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models

from torch.utils.data import DataLoader

In [7]:
json_file = 'dataset_shan.json'
transform = transforms.Compose([
    # transforms.Resize((224, 224)),  
    transforms.ToTensor()
])

dataset = DroneDataset(json_file, cur_folder_path, transform)

In [8]:
num_classes = 5
batch_size = 30

In [9]:
train_set, valid_set = torch.utils.data.random_split(dataset, [13905, 2453])

train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(dataset=valid_set, batch_size=batch_size, shuffle=True)

In [10]:
# resnet34 = torch.hub.load('pytorch/vision:v0.10.0', 'googlenet', pretrained=False)

resnet34 = models.resnet34(pretrained=True) 
resnet34.fc = nn.Linear(resnet34.fc.in_features, num_classes)

# resnet34.load_state_dict(torch.load('resnet34_model.pth'))

/opt/conda/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [22]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
resnet34.to(device)

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [21]:
in_channel = 3
learning_rate = 1e-3
batch_size = 35
num_epochs = 100

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet34.parameters(), lr=learning_rate)

In [23]:
import time
from tqdm import tqdm

start = time.time()

for epoch in tqdm(range(1, num_epochs+1)):
    losses = []
    resnet34.train()
    for batch_idx, (data, targets) in enumerate(train_loader):
        data, targets = data.to(device=device), targets.to(device=device)
        # print(data.shape)
        # print(data.unsqueeze(0).shape)
        scores = resnet34(data)
        loss = criterion(scores, targets)
        
        losses.append(loss.item())
        
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
        
    print(f'Cost at epoch {epoch} is {sum(losses) / len(losses)}')
        
finish = time.time()
print('Finished Training in ', finish-start)

torch.save(resnet34.state_dict(), './resnet34_model.pth')

  0%|          | 0/100 [00:00<?, ?it/s]


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [15]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device=device), y.to(device=device)
            
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
            
    accuracy = num_correct / num_samples
    print(f'Accuracy: {accuracy * 100:.5f}%')

In [16]:
check_accuracy(train_loader, resnet34)

Accuracy: 99.94966%


In [40]:
check_accuracy(valid_loader, resnet34)

NameError: name 'check_accuracy' is not defined